Charger la donnée

In [ ]:
df = spark.read.load("---.parq")
df.show()
# or spark.read.parquet

Afficher le nombre d'éléments du DF

In [ ]:
df.count()

Afficher le type des éléments du DF

In [ ]:
type(df)
# df.schema

Afficher le schéma du DF

In [ ]:
df.printSchema()

Ordonnez les bateaux par vitesse

In [ ]:
df = df.orderBy("speed")
df.show()

Pour disposer d'une date plus lisible, rajoutez la colonne "timestamp_readable"
Pour cela, il vous faudra d'abord transformer le timestamp en secondes (timestamp_pos est en ms)

In [ ]:
df_date = df.withColumn("timestamp_readable", (df['timestamp_pos']/1000).cast("timestamp"))
df_date.show()

Il est possible de créer un nouveau dataframe contenant des statistiques sur un dataframe

In [ ]:
description_df = df.describe()
description_df.show()

Filtrer le dataframe bateaux sur un bateau en particulier

In [ ]:
monBateau = df.filter("id_vessel = 28052")
monBateau.show()

Le nombre de bateaux avec une vitesse == 0

In [ ]:
df.filter(df["speed"]==0).count()

Les types distincts de bateaux

In [ ]:
df[["type"]].dropDuplicates().show()

On affiche la moyenne de la vitesse des bateaux sans tenir compte des bateaux a l'arret

In [ ]:
enmarche = df.filter("speed !=0").groupBy("id_vessel").mean("speed")
enmarche.show()

On affiche la date min puis la date max de notre DF bateaux

In [ ]:
df_date.groupby("id_vessel").agg({"timestamp_readable": "min"}).show()
df_date.groupby("id_vessel").agg({"timestamp_readable": "max"}).show()

Créer un dataframe avec l'id, la vitesse et le timestamp readable

In [ ]:
spd = df_date.select(["id_vessel", "speed", "timestamp_readable"])
spd.show()

Créer une colonne mois pour le mois en cours

In [ ]:
from pyspark.sql.functions import month

spd = spd.withColumn("month", month(df_date["timestamp_readable"]))
spd.show()

Ajouter une colonne boolean à True si le bateau est en mouvement

In [ ]:
spd = spd.withColumn("BooleanCol", spd.speed !=0)
spd.show()

Enregistrer la table

In [ ]:
df.createOrReplaceTempView("save_bateaux")

Afficher par type, le nombre de bateaux, la vitesse max et la moyenne de la vitesse, ordonnés par nombre de bateaux (requete SQL)

In [ ]:
spark.sql("""SELECT type, max(speed), avg(speed), count(*) as num 
FROM save_bateaux 
GROUP BY type 
ORDER BY count(*)""").show()